# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import numpy as np

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/dquan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
ls data

DisasterResponse.db      disaster_response_db.db
categories.csv           messages.csv


In [15]:
# load data from database
engine = create_engine('sqlite:///data/disaster_response_db.db')
df = pd.read_sql_table("disaster_response_db_table", con=engine.connect())

In [16]:
df['related'] = df['related'].map(lambda x: 1 if x == 2 else x)

In [17]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [19]:
X.values[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [20]:
def tokenize(text):
    # lower case
    text = text.lower()
    # strip all punctuation
    text = re.sub(r"[^\w\d]", " ", text)
    text = re.sub(r"\s+", " ", text)
    tokens = nltk.word_tokenize(text)
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
pipeline = Pipeline([
    ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),
        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=41)

In [23]:
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [24]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data


In [94]:
print(classification_report(y_true=y_test.values.astype("int"), y_pred=y_prediction_test.astype("int"), target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5023
               request       0.83      0.37      0.51      1160
                 offer       0.00      0.00      0.00        29
           aid_related       0.74      0.52      0.61      2726
          medical_help       0.65      0.06      0.11       547
      medical_products       0.65      0.07      0.12       305
     search_and_rescue       0.43      0.02      0.03       171
              security       0.00      0.00      0.00       122
              military       0.75      0.06      0.12       232
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.21      0.35       388
                  food       0.80      0.44      0.57       733
               shelter       0.75      0.22      0.35       572
              clothing       0.80      0.08      0.15        97
                 money       0.70      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
# pipeline1.get_params().keys()
parameters_grid = {
              'classifier__estimator__n_estimators': [10, 20, 40]}


cv = GridSearchCV(pipeline, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('text_pipeline',
                                        Pipeline(steps=[('count_vectorizer',
                                                         CountVectorizer(tokenizer=<function tokenize at 0x14c2c8820>)),
                                                        ('tfidf_transformer',
                                                         TfidfTransformer())])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__n_estimators': [10, 20, 40]},
             scoring='f1_micro')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

In [28]:
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5023
               request       0.87      0.44      0.59      1160
                 offer       0.00      0.00      0.00        29
           aid_related       0.78      0.60      0.68      2726
          medical_help       0.64      0.06      0.11       547
      medical_products       0.81      0.06      0.10       305
     search_and_rescue       0.86      0.04      0.07       171
              security       0.00      0.00      0.00       122
              military       0.80      0.07      0.13       232
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.35      0.50       388
                  food       0.84      0.42      0.56       733
               shelter       0.82      0.22      0.34       572
              clothing       0.40      0.02      0.04        97
                 money       0.86      

/Users/dquan/opt/miniconda3/envs/fpt/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dquan/opt/miniconda3/envs/fpt/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dquan/opt/miniconda3/envs/fpt/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [29]:
import pickle
filename = 'models/model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.